In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Options
pd.set_option('precision', 2)
%matplotlib inline

In [ ]:
# CONSTANTS
DOW_ORDER =['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
MAX_MINUTES = 60 * 5

In [ ]:
clean_df = (
    pd.read_csv('data/clean_data.csv')
    .assign(
        day_of_week=lambda d: pd.to_datetime(d.date).dt.dayofweek,
        day_name=lambda d: pd.to_datetime(d.date).dt.day_name(),
        log_SPOSTMIN=lambda d: np.log(d.SPOSTMIN),
        log_SACTMIN=lambda d: np.log(d.SACTMIN),
    )
)

print(clean_df.shape)
clean_df.columns

In [ ]:
open_days_df = clean_df[(clean_df.SPOSTMIN > 0) | (clean_df.SACTMIN > 0)]
actuals_df = open_days_df[(open_days_df.SACTMIN > 0) & (open_days_df.SACTMIN < MAX_MINUTES)]

# Summary and distribution

In [ ]:
(
    open_days_df
    .loc[:, ['short_name', 'duration', 'SACTMIN', 'SPOSTMIN']]
    .groupby(['short_name', 'duration'])
    .describe()
)

## Posted wait-time boxplot per ride

In [ ]:
# Plot the orbital period with horizontal boxes
plt.figure(figsize=(7, 6))
ax = sns.boxplot(x='SPOSTMIN', y='short_name', data=open_days_df,
                 whis=[5, 95], width=.6, palette="vlag")
ax.set(xlabel='Posted wait time (log(minutes))', ylabel='')
ax.set_xscale('log')

# Relationships

## Wait times per day of week

### Actual wait times: biased observation of actuals

In [ ]:
plt.figure(figsize=(7, 6))
g = sns.boxplot(x='day_of_week', y='SACTMIN', data=actuals_df, )
g = g.set(xlabel='Day of week', xticks=range(7), xticklabels=DOW_ORDER,
          ylabel='Actual wait time (log(minutes))', yscale='log')

In [ ]:
g = sns.relplot(x='day_of_week', y='SACTMIN', kind='line', data=actuals_df, height=7, aspect=2)
g = g.set(xlabel='Day of week', xticks=range(7), xticklabels=DOW_ORDER,
          ylabel='Actual wait time (minutes)')

### Posted wait times: complete observation of estimates

In [ ]:
plt.figure(figsize=(7, 6))
g = sns.boxplot(x='day_of_week', y='SPOSTMIN', data=open_days_df, )
g = g.set(xlabel='Day of week', xticks=range(7), xticklabels=DOW_ORDER,
          ylabel='Posted wait time (log(minutes))', yscale='log')

In [ ]:
g = sns.relplot(x='day_of_week', y='SPOSTMIN', kind='line', data=open_days_df, height=7, aspect=2)
g = g.set(xlabel='Day of week', xticks=range(7), xticklabels=DOW_ORDER,
          ylabel='Posted wait time (minutes)')

In [ ]:
seasons_map = (
    open_days_df
    .groupby('SEASON')
    ['date']
    .min()
    .reset_index()
)

# Seasons to exclude, for viz. purposes
exclude_seasons = ['CHRISTMAS PEAK', 'MARDI GRAS', 'JULY 4TH', 'JERSEY WEEK']
seasons_map = seasons_map[~seasons_map.SEASON.isin(exclude_seasons)]
seasons_map.sort_values('date')

In [ ]:
g = sns.relplot(x='date', y='SPOSTMIN', kind='line', data=open_days_df, height=7, aspect=2)
g = g.set(xlabel='Date', ylabel='Posted wait time (minutes)', xticks=seasons_map.date, xticklabels=seasons_map.SEASON)
g = g.set_xticklabels(rotation=45, horizontalalignment='right')

## Wait times by weather

In [ ]:
g = sns.lmplot(x='WDWMEANTEMP', y='log_SPOSTMIN', data=open_days_df,
               col='short_name', col_wrap=3, height=7)